In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import mean, stddev, col, log
from pyspark.sql.functions import to_date, dayofweek, to_timestamp
from pyspark.sql import types 
from pyspark.sql.functions import col, udf
from datetime import datetime
from pyspark.sql.types import DateType
from pyspark.sql.functions import year, month
from pyspark.sql.functions import dayofmonth, weekofyear
from pyspark.sql.functions import split, explode
from pyspark.sql.functions import coalesce, first, lit
from pyspark.ml.feature import Binarizer
from pyspark.ml.feature import Bucketizer
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.sql.functions import regexp_extract, col

In [3]:
from modules.pyspark import CPySpark
from pyspark.sql import Row, SQLContext

In [4]:
spark = CPySpark(session=True, sql=True)

In [5]:
spark = spark.session

In [6]:
df = spark.read.csv("./data/2017_StPaul_MN_Real_Estate.csv", header=True, inferSchema=True)

In [8]:
df.toPandas()

,No,MLSID,StreetNumberNumeric,streetaddress,STREETNAME,PostalCode,StateOrProvince,City,SalesClosePrice,LISTDATE,...,RoomArea7,RoomFloor7,RoomArea8,RoomFloor8,Bedrooms,SQFTBELOWGROUND,AssumableMortgage,AssociationFee,ASSESSMENTPENDING,AssessedValuation
0,1,RMLS,11511,11511 Stillwater Blvd N,Stillwater,55042,MN,LELM - Lake Elmo,143000,7/15/2017 0:00,...,None,None,None,None,3,0,None,0,Unknown,0.0
1,2,RMLS,11200,11200 31st St N,31st,55042,MN,LELM - Lake Elmo,190000,10/9/2017 0:00,...,10x11,Main,11x11,Lower,4,960,None,0,Unknown,0.0
2,3,RMLS,8583,8583 Stillwater Blvd N,Stillwater,55042,MN,LELM - Lake Elmo,225000,6/26/2017 0:00,...,None,None,None,None,2,0,Not Assumable,0,No,0.0
3,4,RMLS,9350,9350 31st St N,31st,55042,MN,LELM - Lake Elmo,265000,8/25/2017 0:00,...,None,None,None,None,2,348,None,0,Unknown,0.0
4,5,RMLS,2915,2915 Inwood Ave N,Inwood,55042,MN,LELM - Lake Elmo,249900,9/12/2017 0:00,...,14x13,Lower,None,None,3,1026,None,0,No,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,RMLS,4933,4933 Sunflower Dr,Sunflower,55129,MN,WB - Woodbury,799000,8/1/2017 0:00,...,12x14,Lower,None,None,3,1686,None,102,No,86.0
4996,4997,RMLS,10746,10746 Watersedge Ln,Watersedge,55129,MN,WB - Woodbury,805000,8/1/2017 0:00,...,14x14,Lower,13x11,Lower,5,2220,None,80,No,0.0
4997,4998,RMLS,10721,10721 Watersedge Ln,Watersedge,55129,MN,WB - Woodbury,865000,3/27/2017 0:00,...,13x12,Upper,15x13,Upper,4,1861,None,86,No,0.0
4998,4999,RMLS,4981,4981 Sunflower Pl,Sunflower,55129,MN,WB - Woodbury,1050216,5/24/2017 0:00,...,13.6x11,Upper,12x11,Upper,4,0,None,105,Unknown,0.0
